In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

logging.langsmith("allforone")

In [1]:
from langgraph.graph import StateGraph, START, END
from agents.state.analysis_state import LocationInsightState
from agents.state.start_state import StartInput
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from utils.util import get_today_str
from utils.llm import LLMProfile
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from prompts import PromptManager, PromptType
from langgraph.prebuilt import ToolNode
from tools.kostat_api import get_move_population, system_prompt
import json
import asyncio


@tool(parse_docstring=False)
def think_tool(reflection: str) -> str:
    """
    [역할]
    당신은 입지/호재 정리 전문가의 내부 반성·점검(Reflection) 담당자입니다.
    최종 보고서에 들어갈 본문(Markdown)을 쓰기 직전에, 데이터 품질·핵심 수치·리스크·보고서용 한 줄 메시지를 짧고 구조적으로 요약해 think_tool에 기록합니다. 이 반성문은 내부용이며, 최종 보고서에 직접 노출되지 않습니다.

    [언제 호출할 것인지]
    - 데이터 수집/정제 → 핵심 수치 산출 → 시계열 해석을 마친 직후 1회 호출(필수)
    - 추가 데이터로 최신 데이터로 바뀌면 갱신 시마다 1회 재호출(선택)

    [강력 지시]
    - 해당 지역에 관련된 내용만 기록
    - 허상 가정,출처 수치 금지
    - 다음 단계(보고서 에이전트)가 바로 쓸 수 있는 한 줄 핵심 메시지 포함

    [나쁜 예]
    - “경제가 좋아진듯함. 분위기 좋음.”(수치·기간·단위·근거 없음)
    - “인근 해운대의 입지지는 이렇다~”(대상 지역 외 서술)
    - “향후 집값 상승 확실.”(근거 없는 단정)

    [검증 체크리스트]
    - 정량 수치가 어긋난 것 이 있는가?
    - GPT가 시계열 판단하기에 좋은 형식으로 되어있는가?
    - 잘못된 내용은 없는가?
    """
    return f"Reflection recorded: {reflection}"


output_key = LocationInsightState.KEY.location_insight_output
start_input_key = LocationInsightState.KEY.start_input
messages_key = LocationInsightState.KEY.messages
target_area_key = StartInput.KEY.target_area
main_type_key = StartInput.KEY.main_type
total_units_key = StartInput.KEY.total_units
web_context_key = LocationInsightState.KEY.web_context
kakao_api_distance_context_key = LocationInsightState.KEY.kakao_api_distance_context
gemini_search_key = LocationInsightState.KEY.gemini_search
perplexity_search_key = LocationInsightState.KEY.perplexity_search


llm = LLMProfile.analysis_llm()
tool_list = [think_tool]
llm_with_tools = llm.bind_tools(tool_list)
tool_node = ToolNode(tool_list)

from tools.gemini_search_tool import gemini_search


def gemini_search_tool(state: LocationInsightState) -> LocationInsightState:
    start_input = state[start_input_key]
    target_area = start_input[target_area_key]
    main_type = start_input[main_type_key]
    total_units = start_input[total_units_key]

    prompt = f"""
    <CONTEXT>
    주소: {target_area}
    규모: {total_units}세대
    타입: {main_type}
    </CONTEXT>
    <GOAL>
    <CONTEXT> 주변 분양호재를 <OUTPUT>을 참조해서 json 형식으로 출력해주세요
    </GOAL>
    <OUTPUT>
    {{
      "분양호재": [
        {{
          "name": "",
          "location": "",
          "description": "",
          "status": ""
        }}
      ]
    }}
    </OUTPUT>
    """
    result = gemini_search(prompt)

    return {gemini_search_key: result}


from tools.perplexity_search_tool import perplexity_search


def perplexity_search_tool(state: LocationInsightState) -> LocationInsightState:
    start_input = state[start_input_key]
    target_area = start_input[target_area_key]
    main_type = start_input[main_type_key]
    total_units = start_input[total_units_key]
    date = get_today_str()

    prompt = f"""
    <CONTEXT>
    주소: {target_area}
    규모: {total_units}세대
    타입: {main_type}
    일시: {date}
    해당 목표일에 주소에서 아파트 분양을 계획하고 있습니다.
    </CONTEXT>

    <GOAL>
    - <CONTEXT>에 나와 있는 스펙으로 해당 사업지에 맞는 핵심 데이터 선별을 위해해 부동산과 관련된 해당지역에 관한 특징을 추려서 출력해주세요.
    - <CONTEXT> 주변 분양호재를 <OUTPUT>을 참조해서 json 형식으로 출력해주세요
    </GOAL>
    <OUTPUT>
    {{
      "해당지역 특징": []
      "분양호재": [
        {{
          "name": "",
          "location": "",
          "description": "",
          "status": ""
        }}
      ]
    }}
    </OUTPUT>
    """
    result = perplexity_search(prompt)
    return {perplexity_search_key: result}


from tools.kakao_api_distance_tool import get_location_profile


def kakao_api_distance_tool(state: LocationInsightState) -> LocationInsightState:
    start_input = state[start_input_key]
    target_area = start_input[target_area_key]
    result = get_location_profile(target_area)
    return {kakao_api_distance_context_key: result}


def analysis_setting(state: LocationInsightState) -> LocationInsightState:
    start_input = state[start_input_key]
    target_area = start_input[target_area_key]
    total_units = start_input[total_units_key]
    main_type = start_input[main_type_key]
    gemini_search = state[gemini_search_key]
    kakao_api_distance_context = state[kakao_api_distance_context_key]
    perplexity_search = state[perplexity_search_key]

    system_prompt = PromptManager(PromptType.LOCATION_INSIGHT_SYSTEM).get_prompt()
    human_prompt = PromptManager(PromptType.LOCATION_INSIGHT_HUMAN).get_prompt(
        target_area=target_area,
        total_units=total_units,
        main_type=main_type,
        date=get_today_str(),
        gemini_search=gemini_search,
        kakao_api_distance_context=kakao_api_distance_context,
        perplexity_search=perplexity_search,
    )
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=human_prompt),
    ]
    return {**state, messages_key: messages}


def agent(state: LocationInsightState) -> LocationInsightState:
    messages = state.get(messages_key, [])
    response = llm_with_tools.invoke(messages)
    new_messages = messages + [response]
    new_state = {**state, messages_key: new_messages}
    new_state[output_key] = response.content
    return new_state


def router(state: LocationInsightState):
    messages = state[messages_key]
    last_ai_message = messages[-1]
    if last_ai_message.tool_calls:
        return "tools"
    return "__end__"


web_search_key = "web_search"
analysis_setting_key = "analysis_setting"
tools_key = "tools"
agent_key = "agent"
gemini_search_key = "gemini_search"
kakao_api_distance_key = "kakao_api_distance"
perplexity_search_key = "perplexity_search"

graph_builder = StateGraph(LocationInsightState)

graph_builder.add_node(gemini_search_key, gemini_search_tool)
graph_builder.add_node(kakao_api_distance_key, kakao_api_distance_tool)
graph_builder.add_node(analysis_setting_key, analysis_setting)
graph_builder.add_node(perplexity_search_key, perplexity_search_tool)

graph_builder.add_node(tools_key, tool_node)
graph_builder.add_node(agent_key, agent)

graph_builder.add_edge(START, gemini_search_key)
graph_builder.add_edge(START, kakao_api_distance_key)
graph_builder.add_edge(START, perplexity_search_key)


graph_builder.add_edge(gemini_search_key, analysis_setting_key)
graph_builder.add_edge(kakao_api_distance_key, analysis_setting_key)
graph_builder.add_edge(perplexity_search_key, analysis_setting_key)
graph_builder.add_edge(analysis_setting_key, agent_key)

graph_builder.add_conditional_edges(agent_key, router, [tools_key, END])
graph_builder.add_edge(tools_key, agent_key)

location_insight_graph = graph_builder.compile()

In [2]:
invoke = await location_insight_graph.ainvoke(
    {
        "start_input": {
            "target_area": "서울 강남구 역삼동",
            "total_units": "1000세대",
            "main_type": "84제곱미터",
        }
    }
)

In [3]:
print(invoke)

{'start_input': {'target_area': '서울 강남구 역삼동', 'total_units': '1000세대', 'main_type': '84제곱미터'}, 'location_insight_output': '{\n  "입지정보": {\n    "교육환경": {\n      "최근접초등학교": {\n        "이름": "서울역삼초등학교",\n        "주소": "서울 강남구 강남대로66길 21",\n        "거리(미터)": 247,\n        "도보예상시간(분)": 4,\n        "통학로위험요소": "강남대로 횡단 필요, 대로변 인접(신호등 및 인도 확보)"\n      },\n      "기타학교": [\n        {\n          "이름": "서울서이초등학교",\n          "거리(미터)": 751\n        },\n        {\n          "이름": "서운중학교",\n          "거리(미터)": 761\n        }\n      ],\n      "학원가": [\n        {\n          "이름": "강남대성학원",\n          "거리(미터)": 196\n        },\n        {\n          "이름": "김영편입학원 강남단과캠퍼스",\n          "거리(미터)": 274\n        }\n      ]\n    },\n    "교통": {\n      "지하철": [\n        {\n          "노선": "신분당선",\n          "역명": "강남역",\n          "출구까지거리(미터)": 459,\n          "도보예상시간(분)": 7\n        },\n        {\n          "노선": "2호선",\n          "역명": "강남역",\n          "출구까지거리(미터)": 540,\n          "도보예상시간(분)": 8\n        },\

In [4]:
print(invoke["location_insight_output"])

{
  "입지정보": {
    "교육환경": {
      "최근접초등학교": {
        "이름": "서울역삼초등학교",
        "주소": "서울 강남구 강남대로66길 21",
        "거리(미터)": 247,
        "도보예상시간(분)": 4,
        "통학로위험요소": "강남대로 횡단 필요, 대로변 인접(신호등 및 인도 확보)"
      },
      "기타학교": [
        {
          "이름": "서울서이초등학교",
          "거리(미터)": 751
        },
        {
          "이름": "서운중학교",
          "거리(미터)": 761
        }
      ],
      "학원가": [
        {
          "이름": "강남대성학원",
          "거리(미터)": 196
        },
        {
          "이름": "김영편입학원 강남단과캠퍼스",
          "거리(미터)": 274
        }
      ]
    },
    "교통": {
      "지하철": [
        {
          "노선": "신분당선",
          "역명": "강남역",
          "출구까지거리(미터)": 459,
          "도보예상시간(분)": 7
        },
        {
          "노선": "2호선",
          "역명": "강남역",
          "출구까지거리(미터)": 540,
          "도보예상시간(분)": 8
        },
        {
          "노선": "2호선",
          "역명": "역삼역",
          "출구까지거리(미터)": 661,
          "도보예상시간(분)": 10
        }
      ],
      "버스": {
        "주요정류장거리(미터)": 

In [5]:
print(invoke["gemini_search"])

```json
{
  "분양호재": [
    {
      "name": "래미안 원펜타스 (신반포15차 재건축)",
      "location": "서울 서초구 반포동 12",
      "description": "총 641세대 중 292세대를 일반분양하는 후분양 단지. 강남 핵심 입지에 위치하여 높은 관심이 예상됨.",
      "status": "2024년 분양 예정"
    },
    {
      "name": "디에이치 방배 (방배5구역 재개발)",
      "location": "서울 서초구 방배동 946-8",
      "description": "총 3,065세대의 대단지 브랜드 아파트로, 강남권의 대규모 공급으로 주목받는 단지.",
      "status": "2024년 분양 예정"
    },
    {
      "name": "압구정 아파트지구 재건축",
      "location": "서울 강남구 압구정동 일대",
      "description": "압구정 2~5구역 등 총 1만여 세대 규모의 초대형 재건축 사업으로, 서울의 대표적인 장기 개발 호재.",
      "status": "사업 추진 중 (조합 설립 및 시공사 선정 단계)"
    }
  ]
}
```


In [ ]:
print(invoke['perplexity_search'])

<think>
사용자는 서울 강남구 역삼동에서 1000세대 규모, 84제곱미터 타입의 아파트 분양이 2025년 11월 4일에 계획되어 있다고 가정하고 있습니다. 

사용자의 요청:
1. 해당 지역(강남구 역삼동)의 부동산 관련 특징 추출
2. 주변 분양호재를 JSON 형식으로 출력

검색 결과를 보면:
- 역삼은하수 재건축: 강남구 역삼동 758번지, 237세대, GS건설 시공, 2025년 6월 분양 예정, 일반분양 74세대
- 이는 강남역, 역삼역 도보권
- 수인분당선 한티역 인근
- 영동근린공원, 이마트 등 인프라

해당 지역 특징:
1. 강남 중심 입지
2. 우수한 교통 환경 (강남역, 역삼역 도보권)
3. 학군 우수
4. 우수한 생활 인프라
5. 직주근접 입지
6. 높은 전세수요

분양호재:
- 역삼은하수 재건축 (237세대, 74세대 일반분양, 2025년 6월)
- 오티에르반포 (서초구, 251가구, 87가구 일반분양, 2025년 예정)


{
    "response_format": "JSON 형식으로 제공"
}
</think>

```json
{
  "해당지역 특징": [
    "강남 업무지구 중심 입지로 높은 직장인 수요",
    "강남역과 역삼역 도보권의 우수한 교통 접근성",
    "수인분당선 한티역 인근 위치",
    "우수한 학군 환경 (역삼중 도보권)",
    "이마트, 영동근린공원 등 풍부한 생활 인프라",
    "1인가구 및 직장인 수요가 높은 지역",
    "강남권 재건축 프리미엠 기대 가능한 입지",
    "전세 수요가 많고 가격이 비교적 안정적인 지역"
  ],
  "분양호재": [
    {
      "name": "역삼은하수 재건축",
      "location": "서울 강남구 역삼동 758번지",
      "description": "GS건설 시공, 237세대 규모 (4개동, 3층~15층), 일반분양 74세대, 강남역·역삼역·한티역 도보권, 영동근린공원 및 이마트 인접",
      "status": 

: 